In [1]:
import csv
import tensorflow as tf
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime

In [2]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"

with open('Interviewdata_Question.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    questions = [nltk.word_tokenize(x[0].lower()) for x in reader]
    print(len(questions))
with open('Interviewdata_Answer.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    answers = [nltk.word_tokenize(x[0].lower()) for x in reader]
    print(len(answers))
with open('Interviewdata_label.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    labels = np.array(list(itertools.chain(*[x for x in reader]))).astype(np.int32)
    print(len(labels))

with open('Interviewtestdata_Question.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    testquestions = [nltk.word_tokenize(x[0].lower()) for x in reader]
    print(len(testquestions))

with open('Interviewtestdata_Answer.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    testanswers = [nltk.word_tokenize(x[0].lower()) for x in reader]
    print(len(testanswers))  

204
204
204
4
4


In [3]:
completedata = questions.copy()
for i in answers:
    completedata.append(i)

In [4]:
len(completedata)

408

In [5]:
len(questions)

204

In [6]:
import pickle
with open('questions.txt', 'wb') as f:
    pickle.dump(completedata, f)

In [7]:
import pickle
with open ('words.txt', 'rb') as fp:
    wordlist = pickle.load(fp)

In [8]:
with open('embeddings.txt','rb') as fp:
    embeddings = pickle.load(fp)

In [9]:
word_to_index = dict([(w,i) for i,w in enumerate(wordlist)])

In [10]:
for i, sent in enumerate(questions):
    questions[i] = [w if w in word_to_index else unknown_token for w in sent]

In [11]:
for i, sent in enumerate(answers):
    answers[i] = [w if w in word_to_index else unknown_token for w in sent]

In [12]:
dummy = 'a'

In [13]:
for i, sent in enumerate(testquestions):
    testquestions[i] = [w if w in word_to_index else dummy for w in sent]

In [14]:
for i, sent in enumerate(testanswers):
    testanswers[i] = [w if w in word_to_index else dummy for w in sent]

In [15]:
import numpy as np
X_question = np.asarray([[word_to_index[w] for w in sent] for sent in questions])
y_answer = np.asarray([[word_to_index[w] for w in sent] for sent in answers])

In [16]:
x_example, y_example = X_question[1], y_answer[1]
print("x:\n%s\n%s" % (" ".join([wordlist[x] for x in x_example]), x_example))
print("\ny:\n%s\n%s" % (" ".join([wordlist[x] for x in y_example]), y_example))

x:
what do you mean by servlet ?
[12, 57, 43, 288, 26, 31, 7]

y:
a servlet is a java program that runs within a web server and serves the client request at runtime.it can receive and respond to request from web clients ( web browser ) .
[8, 31, 2, 8, 29, 184, 28, 362, 89, 8, 87, 58, 9, 713, 0, 84, 52, 70, 1474, 13, 559, 9, 1434, 4, 52, 35, 87, 1238, 11, 87, 185, 10, 1]


In [17]:
X_testquestion = np.asarray([[word_to_index[w] for w in sent] for sent in testquestions])
y_testanswer = np.asarray([[word_to_index[w] for w in sent] for sent in testanswers])

In [18]:
questionlength = [len(i) for i in X_question]
answerlength = [len(i) for i in y_answer]

In [19]:
testquestionlength = [len(i) for i in X_testquestion]
testanswerlength = [len(i) for i in y_testanswer]

In [20]:
import numpy as np
x = X_question
y = y_answer
max_length = max(len(row) for row in y)
x_padded = np.array([row + [0] * (max_length - len(row)) for row in x])

max_length = max(len(row) for row in y)
y_padded = np.array([row + [0] * (max_length - len(row)) for row in y])


p = X_testquestion
q = y_testanswer
max_length = max(len(row) for row in p)
x_testpadded = np.array([row + [0] * (max_length - len(row)) for row in p])

max_length = max(len(row) for row in p)
y_testpadded = np.array([row + [0] * (max_length - len(row)) for row in q])

In [21]:
len(x_padded), len(y_padded), len(x_padded[1]), len(y_padded[1])

(204, 204, 164, 164)

In [22]:
len(x_testpadded), len(y_testpadded), len(x_testpadded[1]), len(y_testpadded[1])

(4, 4, 21, 21)

In [23]:
len(wordlist), len(embeddings)

(1719, 1719)

In [24]:
tf.InteractiveSession().run(tf.concat([[[1, 2, 3], [4, 5, 6]],[[7, 8, 9], [10, 11, 12]]],0))

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [25]:
with open('Interviewtestdata_label.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    testlabels = np.array(list(itertools.chain(*[x for x in reader]))).astype(np.int32)
    print(len(testlabels))

4


In [26]:
testlabels

array([0, 0, 0, 0])

In [27]:
tf.reset_default_graph()

from tensorflow.contrib import rnn 
hm_epochs = 1
batch_size = 4
state_size = 200
vocab_size = 1719

# Placeholders
context = tf.placeholder(tf.int32, [batch_size, None]) # [batch_size, num_steps]
context_len = tf.placeholder(tf.int32, [batch_size])
utterance = tf.placeholder(tf.int32, [batch_size,None])
utterance_len = tf.placeholder(tf.int32, [batch_size])
targets = tf.placeholder(tf.int32, [batch_size])
target = tf.placeholder(tf.int32, [batch_size])

def next_batch(step):
    p,q = x_padded[batch_size*step:batch_size*(step+1)], y_padded[batch_size*step:batch_size*(step+1)]
    r = np.array(questionlength[batch_size*step:batch_size*(step+1)])
    s = np.array(answerlength[batch_size*step:batch_size*(step+1)])
    t = labels[batch_size*step:batch_size*(step+1)]
    return p,q,r,s,t

def test_batch(step):
    a,b = x_testpadded[batch_size*step:batch_size*(step+1)], y_testpadded[batch_size*step:batch_size*(step+1)]
    c = np.array(testquestionlength[batch_size*step:batch_size*(step+1)])
    d = np.array(testanswerlength[batch_size*step:batch_size*(step+1)])
    e = testlabels[batch_size*step:batch_size*(step+1)]
    return a,b,c,d,e
        
def dual_encoder_model(context,context_len,utterance,utterance_len,targets):
    
    #Embedding the question and answer
    context_embedded = tf.nn.embedding_lookup(embeddings, context, name="embed_context")
    utterance_embedded = tf.nn.embedding_lookup(embeddings, utterance, name="embed_utterance")
    
    print(" This is the embeddings for both Q&A:", context_embedded, utterance_embedded)
    
    #initial_state = tf.tile(tf.get_variable('initial_state', [1, state_size],
                     #   initializer=tf.constant_initializer(0.0)),[batch_size,1])
    #this is thr RNN model 
    with tf.variable_scope("rnn") as vs:
        cell = tf.contrib.rnn.LSTMCell(state_size)
        
    #add a dropout layer to prevent overfitting
    cell = tf.contrib.rnn.DropoutWrapper(cell=cell, output_keep_prob = 0.5)
    #feed inputs to dynamic rnn for training    
    print("These are the inputs to RNN:", tf.concat([context_embedded, utterance_embedded],0))
    
    rnn_outputs, rnn_states = tf.nn.dynamic_rnn(
                                        cell,
                                        tf.concat([context_embedded, utterance_embedded],0),
                                        sequence_length=tf.concat([context_len, utterance_len],0),
                                        dtype=tf.float32)
    print("These are the outputs from RNN:", rnn_outputs, "This are the states outputs:", rnn_states)
    
    encoding_context, encoding_utterance = tf.split(rnn_states.h,2,0)
    
    print("This is the encoding context:", encoding_context)
    print("This is the encoding utterance:", encoding_utterance)
    
    with tf.variable_scope("prediction") as vs:
        M = tf.get_variable("M",
          shape=[state_size, state_size],
          initializer=tf.truncated_normal_initializer())

    # "Predict" a  response: c * M
    generated_response = tf.matmul(encoding_context, M)
    print("This is the generated response:", generated_response)
    generated_response = tf.expand_dims(generated_response, 2)
    print("This is the generated response:", generated_response)
    encoding_utterance = tf.expand_dims(encoding_utterance, 2)
    print("This is the encoding utterance:", encoding_utterance)
    # Dot product between generated response and actual response
    # (c * M) * r
    logit = tf.matmul(generated_response, encoding_utterance, True)
    print("This is the logits before squeeze:", logit)
    logits = tf.squeeze(logit, [2])
    print("This is the logits after squeeze:", logits)

    # Apply sigmoid to convert logits to probabilities
    probs = tf.sigmoid(logits)   
    print("This is the probability after sigmoid:", probs)
    return tf.reshape(logits, [-1]), tf.reshape(probs, [-1])

def train_encoder(x):
    prediction, probs = dual_encoder_model(context,context_len,utterance,utterance_len,targets)
    # Calculate the binary cross-entropy loss
    print("This is the prediction size:", prediction)
    print("This is the target size:", targets)
    losses = tf.nn.sigmoid_cross_entropy_with_logits(logits = prediction, labels = tf.to_float(targets))

    # Mean loss across the batch of examples
    cost = tf.reduce_mean(losses, name="cost")
    optimizer = tf.train.AdamOptimizer().minimize(cost)   
    saver = tf.train.Saver()
        
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        for epoch in range(hm_epochs):
            epoch_loss = 0 
            for step in range(int(len(x_padded)/batch_size)):
                ques, ans, queslength, anslength, labl = next_batch(step)
                _, c, logts, prob = sess.run([optimizer, cost, prediction, probs], feed_dict={context: ques, 
                                                              context_len: queslength,
                                                              utterance: ans, 
                                                              utterance_len: anslength, 
                                                              targets: labl})
                epoch_loss += c
                #print('step:', step, 'logits:', logts, 'probability:', prob)
            print('***************************** Epoch Completed *************************************')
            print('Epoch:', epoch, 'completed out of:',hm_epochs,'loss:',epoch_loss, 'probability:', prob)
        save_path = saver.save(sess, "./Interviewmodel/")
        print("Model saved in file: %s" % save_path)  
        qe, an, quelength, anlength, lbl = test_batch(0)
        correct = tf.equal(tf.cast(probs,tf.int32),target)
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        Accurate,truths,probas = sess.run([accuracy,correct,probs], feed_dict={context: qe, 
                                                              context_len: quelength,
                                                              utterance: an, 
                                                              utterance_len: anlength, 
                                                              target: lbl})
        print('The Accuracy is:', Accurate, "Truths:", truths, "Probs:", probas)

In [28]:
train_encoder(x)

 This is the embeddings for both Q&A: Tensor("embed_context:0", shape=(4, ?, 200), dtype=float32) Tensor("embed_utterance:0", shape=(4, ?, 200), dtype=float32)
These are the inputs to RNN: Tensor("concat:0", shape=(8, ?, 200), dtype=float32)
These are the outputs from RNN: Tensor("rnn_1/transpose:0", shape=(8, ?, 200), dtype=float32) This are the states outputs: LSTMStateTuple(c=<tf.Tensor 'rnn_1/while/Exit_2:0' shape=(?, 200) dtype=float32>, h=<tf.Tensor 'rnn_1/while/Exit_3:0' shape=(?, 200) dtype=float32>)
This is the encoding context: Tensor("split:0", shape=(?, 200), dtype=float32)
This is the encoding utterance: Tensor("split:1", shape=(?, 200), dtype=float32)
This is the generated response: Tensor("MatMul:0", shape=(?, 200), dtype=float32)
This is the generated response: Tensor("ExpandDims:0", shape=(?, 200, 1), dtype=float32)
This is the encoding utterance: Tensor("ExpandDims_1:0", shape=(?, 200, 1), dtype=float32)
This is the logits before squeeze: Tensor("MatMul_1:0", shape=(?